# HinglishSarc - Exploratory Data Analysis (EDA)

**Week 1, Day 1-2: Environment Setup & Dataset Exploration**

This notebook explores the three datasets:
1. Sarcasm Dataset (9,593 samples)
2. Emotion Dataset (25,688 samples)
3. MLT Dataset (30,000 samples)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import emoji
import re
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✓ Imports successful!')

## 1. Load Datasets

In [ ]:
# Load datasets
sarc_df = pd.read_csv('../data/raw/sarcasm_hinghlish_dataset.csv')
emotion_df = pd.read_excel('../data/raw/emotion_hinghlish_dataset.xlsx')
mlt_df = pd.read_csv('../data/raw/mlt_hinghlish_dataset.csv')

print('Sarcasm Dataset:', sarc_df.shape)
print('Emotion Dataset:', emotion_df.shape)
print('MLT Dataset:', mlt_df.shape)

## 2. Sarcasm Dataset Analysis

In [ ]:
print('=== SARCASM DATASET ===')
print(f'Shape: {sarc_df.shape}')
print(f'Columns: {sarc_df.columns.tolist()}')
print(f'\nFirst 5 samples:')
sarc_df.head()

In [ ]:
# Label distribution
label_counts = sarc_df['label'].value_counts().sort_index()
print('Label Distribution:')
print(label_counts)
print(f'\nSarcasm ratio: {sarc_df["label"].mean():.2%}')

# Visualize
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
label_counts.plot(kind='bar', ax=ax[0], color=['#3498db', '#e74c3c'])
ax[0].set_title('Sarcasm Label Distribution')
ax[0].set_xlabel('Label')
ax[0].set_ylabel('Count')
ax[0].set_xticklabels(['Non-Sarcastic (0)', 'Sarcastic (1)'], rotation=0)

# Pie chart
ax[1].pie(label_counts.values, labels=['Non-Sarcastic', 'Sarcastic'], 
          autopct='%1.1f%%', colors=['#3498db', '#e74c3c'])
ax[1].set_title('Sarcasm Distribution')

plt.tight_layout()
plt.savefig('../outputs/figures/sarcasm_label_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Text length analysis
sarc_df['text_length'] = sarc_df['text'].str.len()
sarc_df['word_count'] = sarc_df['text'].str.split().str.len()

print('Text Length Statistics:')
print(sarc_df.groupby('label')[['text_length', 'word_count']].describe())

In [ ]:
# Visualize text length distribution
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Character length
sarc_df[sarc_df['label']==0]['text_length'].hist(ax=ax[0], bins=50, alpha=0.6, 
                                                   label='Non-Sarcastic', color='#3498db')
sarc_df[sarc_df['label']==1]['text_length'].hist(ax=ax[0], bins=50, alpha=0.6, 
                                                   label='Sarcastic', color='#e74c3c')
ax[0].set_title('Text Length Distribution')
ax[0].set_xlabel('Character Count')
ax[0].set_ylabel('Frequency')
ax[0].legend()
ax[0].set_xlim(0, 500)

# Word count
sarc_df[sarc_df['label']==0]['word_count'].hist(ax=ax[1], bins=50, alpha=0.6, 
                                                 label='Non-Sarcastic', color='#3498db')
sarc_df[sarc_df['label']==1]['word_count'].hist(ax=ax[1], bins=50, alpha=0.6, 
                                                 label='Sarcastic', color='#e74c3c')
ax[1].set_title('Word Count Distribution')
ax[1].set_xlabel('Word Count')
ax[1].set_ylabel('Frequency')
ax[1].legend()
ax[1].set_xlim(0, 80)

plt.tight_layout()
plt.savefig('../outputs/figures/sarcasm_text_length.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Emoji Analysis

In [ ]:
# Extract emojis
def extract_emojis(text):
    return [c for c in str(text) if c in emoji.EMOJI_DATA]

sarc_df['emoji_count'] = sarc_df['text'].apply(lambda x: len(extract_emojis(str(x))))

print('Emoji Usage Statistics:')
print(sarc_df.groupby('label')['emoji_count'].describe())

print(f"\nSamples with emojis: {(sarc_df['emoji_count'] > 0).sum()} ({(sarc_df['emoji_count'] > 0).mean():.2%})")

## 4. Code-Mixing Analysis

In [ ]:
# Simple code-mixing detection (Hindi/Devanagari script)
def contains_hindi(text):
    # Devanagari Unicode range: U+0900 to U+097F
    hindi_pattern = re.compile(r'[\u0900-\u097F]')
    return bool(hindi_pattern.search(str(text)))

sarc_df['has_hindi'] = sarc_df['text'].apply(contains_hindi)

print('Code-Mixing Statistics:')
print(f'Samples with Hindi script: {sarc_df["has_hindi"].sum()} ({sarc_df["has_hindi"].mean():.2%})')
print(f'\nBy Label:')
print(sarc_df.groupby('label')['has_hindi'].value_counts())

## 5. Emotion Dataset Analysis

In [ ]:
print('=== EMOTION DATASET ===')
print(f'Shape: {emotion_df.shape}')
print(f'Columns: {emotion_df.columns.tolist()}')
print(f'\nEmotion Distribution:')
emotion_counts = emotion_df['emotion'].value_counts()
print(emotion_counts)

In [ ]:
# Visualize emotion distribution
plt.figure(figsize=(12, 6))
emotion_counts.plot(kind='bar', color='#9b59b6')
plt.title('Emotion Dataset - Emotion Distribution')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../outputs/figures/emotion_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Sample Texts

In [ ]:
print('=== SAMPLE SARCASTIC TEXTS ===')
for idx, row in sarc_df[sarc_df['label']==1].sample(5, random_state=42).iterrows():
    print(f'\n{idx}: {row["text"]}')

In [ ]:
print('=== SAMPLE NON-SARCASTIC TEXTS ===')
for idx, row in sarc_df[sarc_df['label']==0].sample(5, random_state=42).iterrows():
    print(f'\n{idx}: {row["text"]}')

## 7. Summary Statistics

In [ ]:
summary = {
    'Dataset': ['Sarcasm', 'Emotion', 'MLT'],
    'Samples': [len(sarc_df), len(emotion_df), len(mlt_df)],
    'Classes': [2, 10, 10],
    'Avg Text Length': [
        sarc_df['text_length'].mean(),
        emotion_df['text'].str.len().mean(),
        mlt_df['hinglish_genz_text'].str.len().mean()
    ]
}

summary_df = pd.DataFrame(summary)
print('\n=== DATASET SUMMARY ===')
print(summary_df)

## ✅ Day 1-2 Checkpoint

**Completed:**
- ✓ Environment setup
- ✓ Dataset loading
- ✓ Exploratory data analysis
- ✓ Text length analysis
- ✓ Emoji analysis
- ✓ Code-mixing analysis

**Key Findings:**
1. Sarcasm dataset has 57.79% sarcasm ratio (mild imbalance)
2. Emotion dataset is fairly balanced across 10 classes
3. Code-mixing is present but low Devanagari script usage
4. Emoji usage varies between classes

**Next Steps:** Day 3-4 - Data Preprocessing